**Google Drice 掛載**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**套件匯入**

import 文字轉向量與預處理套件

匯入自訓練模型(根據爬蟲下來的文本)

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models import FastText
import pandas as pd 
import re ,nltk
from nltk.tokenize import regexp_tokenize,sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords  
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  
fname = get_tmpfile("/content/drive/MyDrive/aws_聊天機器人/AWS_CORPUS_fasttext3.model")
model = FastText.load(fname)

stop_words = set(stopwords.words('english'))  
# import gensim.downloader
# model = gensim.downloader.load('fasttext-wiki-news-subwords-300')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


讀取根據AWS Blog FQA Support 所爬下來的文本

In [ ]:
#from nltk.corpus import stopwords
nltk.download('punkt') 
nltk.download('wordnet')
wtlem = WordNetLemmatizer()
from nltk.tokenize import word_tokenize
# nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))  
# print(word_tokenize(text))
SUPPORT = pd.read_json("/content/drive/MyDrive/aws_聊天機器人/final_support_data.json").dropna()
BLOG = pd.read_json("/content/drive/MyDrive/aws_聊天機器人/aws_blog_data.json").dropna()
FQA = pd.read_json("/content/drive/MyDrive/aws_聊天機器人/Faq_corpus2.json").dropna()

stop_words.add("i")
drop_list = {"about","above","after","before","after","between","be","do","for","have","can","where","which","how","what","why","who","when","because","q"}
stop_words = set.difference(stop_words,drop_list)
print(stop_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


{"it's", 'each', "don't", 've', "isn't", 'or', 'hers', "hasn't", "you're", "mightn't", 'herself', 'too', "didn't", 'no', 'shan', 'yourself', 'd', 'in', 're', 'any', "couldn't", "you'll", 'doing', 'own', 'you', 'theirs', "should've", 'couldn', "wasn't", 'while', 'such', 'wouldn', 'does', 'it', "haven't", 'ourselves', 'were', 'that', 'this', 'being', 'once', 'on', 'now', 'of', "that'll", 'was', 'again', 'nor', 'than', "doesn't", 'hasn', 'whom', 'other', 'shouldn', "mustn't", 'they', 'aren', 'as', 'here', 'all', 'into', "hadn't", 'didn', 'up', 'don', 'ain', 'just', "you've", 'under', 'am', 'has', 'over', 'doesn', 'by', 'haven', 'mustn', 'ours', 'isn', 'from', "won't", 'needn', 'their', 'himself', 'will', "aren't", 'with', 'ma', 'wasn', 'o', 'me', 'had', 'yours', "needn't", 'your', 'did', 'those', 'out', 'then', 'there', 'against', 'through', "shouldn't", 'themselves', 'same', 'mightn', 'and', 'should', 'll', 'my', 'she', 'having', 'them', 'off', 'only', "wouldn't", 't', "weren't", 'are', 

文字預處理、刪除標點符號

文字預處理包含:英文正則表達、刪除停用詞、字根還原

In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def preprocess_sent(new_sent):
  new_sent = new_sent.lower() 
  sent_stayWord = re.sub(r'[^\w\s]','',new_sent)                #去除符號 數字資料及本身已經刪除 
  corpus_tokenize = regexp_tokenize(sent_stayWord, pattern='\w+|\$[\d\.]+|\S+')
  for i in range(len(corpus_tokenize)):                                             
    corpus_tokenize[i] = wtlem.lemmatize(corpus_tokenize[i],'v')        #wordnet
  # print(corpus_tokenize)
  filtered_sentence = [w for w in corpus_tokenize if not w in stop_words]  #刪除停用詞
  return filtered_sentence


預處理結果顯示

In [ ]:
nltk.download('omw-1.4')
preprocess_sent("i are good boy")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


['be', 'good', 'boy']

處理好的資料集合併

In [ ]:
all_Title = pd.concat([SUPPORT[['Title','url']],BLOG[['Title','url']]],axis=0).reset_index()
get_vec_title =[]
# for ti in all_Title['Title']:
#   print(preprocess_sent(ti))
all_Title

進行Fasttext編碼

In [ ]:
'''
def get_embedding_from_fast_text(corpus_title , model_): #包含preprocess
  all_sent_vec =[]
  for ti in corpus_title:
    ti_pre = preprocess_sent(ti)
    sent_vec = []
    for i in ti_pre:
      sent_vec.append(model_.wv[i]) 
      print(i)
    all_sent_vec.append(sum(sent_vec)/len(ti_pre))
  return all_sent_vec
'''

def get_embedding_from_single_sent(sent_,model_):
  sent_vec = []
  ti_pre = preprocess_sent(sent_)
  for i in ti_pre:
    # print(ti_pre)

    sent_vec.append(model.wv[i]) 
  #print(sent_vec[0])
  return (sum(sent_vec)/len(ti_pre))

def get_embedding_from_df(df , model_):
  vector = []
  #print(i)
  for i in df :
    #print(i)
    vector.append(get_embedding_from_single_sent(i,model_))
  return vector


all_title_embedding = get_embedding_from_df(all_Title['Title'],model)
all_qa_embedding = get_embedding_from_df(FQA['question'],model)

In [ ]:
all_Title.insert(2,"Embedding",all_title_embedding)
FQA.insert(2,"Embedding",all_qa_embedding)
#all_title_embedding[0]

儲存處理過後的結果

In [ ]:
# pd.to_json()
all_Title.to_json("/content/drive/MyDrive/aws_聊天機器人/For_demo/all_title3.json")
FQA.to_json("/content/drive/MyDrive/aws_聊天機器人/For_demo/FQA3.json")

可以開始問問題囉!

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics.pairwise import cosine_similarity
#from scipy.spatial.distance import cdist
qa = "How does AWS Auto Scaling discover what resources can scale?"
q = get_embedding_from_single_sent(qa , model).reshape(1,50)
# cosine_similarity(x, y).sum()
cos_list =[]
cos_list_qa = [ ]
for i in range(len(all_Title)-1) :
  blog = all_Title['Embedding'].values[i].reshape(1,50)
  FQ_A = FQA['Embedding'].values[i].reshape(1,50)
  #print(cosine_similarity(q, a))
  cos_list.append(cosine_similarity(q, blog))
  cos_list_qa.append(cosine_similarity(q, FQ_A))

sorted_cos_list  = sorted(cos_list)
sorted_cos_list_qa = sorted(cos_list_qa)

for i in range(1,3):
  print("您的問題可能是 \n",FQA.question[cos_list_qa.index(sorted_cos_list_qa[-i])])
  print("解答為  \n A: ",remove_html_tags(FQA.answer[cos_list_qa.index(sorted_cos_list_qa[-i])]))
  print("您可以參考此Blog的連結-1",all_Title.Title[cos_list.index(sorted_cos_list[-i])])
  print(all_Title.url[cos_list.index(sorted_cos_list[-i])])
  print("您可以參考此Blog的連結-2",all_Title.Title[cos_list.index(sorted_cos_list[-i-2])])
  print(all_Title.url[cos_list.index(sorted_cos_list[-i-2])])
  print("------------------------------------------------------------------------------")

# print("您的問題可能是 \n",FQA.question[cos_list_qa.index(max(cos_list_qa))])
# print("解答為  \n A: ",FQA.answer[cos_list_qa.index(max(cos_list_qa))])
# print("您可以參考此Blog的連結-1",all_Title.Title[cos_list.index(max(cos_list))])
# print(all_Title.url[cos_list.index(max(cos_list))])
# print("您可以參考此Blog的連結-2",all_Title.Title[cos_list.index(max(cos_list))])
# print(all_Title.url[cos_list.index(max(cos_list))])


您的問題可能是 
 Q. How do I select an application stack within AWS Auto Scaling?
解答為  
 A:  You can either select an AWS CloudFormation stack or select resources based on common resource tag(s). Please note that currently, ECS services cannot be discovered using tags. 


 Show less






 
您可以參考此Blog的連結-1 How do I resolve issues with DynamoDB auto scaling?
https://aws.amazon.com//premiumsupport/knowledge-center/dynamodb-auto-scaling/
您可以參考此Blog的連結-2 How do I delay Auto Scaling termination of unhealthy Amazon EC2 instances so I can troubleshoot them?
https://aws.amazon.com//premiumsupport/knowledge-center/auto-scaling-delay-termination/
------------------------------------------------------------------------------
您的問題可能是 
 Q. What can I scale with AWS Auto Scaling?
解答為  
 A:  You can use AWS Auto Scaling to setup scaling for the following resources in your application through a single, unified interface:

Amazon EC2 Auto Scaling groups
Amazon Elastic Container Service (ECS) services (current

In [ ]:

# print("q 為 : ",str(all_Title.Title[ind]))
print("問題為 : " , qa)
print("index 為 : " ,str(cos_list.index(max(cos_list))))
print("cosine 為 :" , str(max(cos_list)))
# print("兩者之間的cosine : ", str(cos_list[ind]))
print("Bot 回傳的 :", str(all_Title.Title[cos_list.index(max(cos_list))]))

問題為 :  How does AWS Auto Scaling discover what resources can scale?
index 為 :  92
cosine 為 : [[0.9077897]]
Bot 回傳的 : How do I ensure that Auto Scaling lifecycle actions complete successfully when I use AWS CodeDeploy?


In [ ]:
remove_html_tags(FQA.answer[cos_list_qa.index(sorted_cos_list_qa[-2])])

'You can use AWS Auto Scaling to setup scaling for the following resources in your application through a single, unified interface:\n\nAmazon EC2 Auto Scaling groups\nAmazon Elastic Container Service (ECS) services (currently ECS services cannot be discovered using resource tags) \nAmazon EC2 Spot Fleets\nAmazon DynamoDB throughput capacity\nAurora replicas for Amazon Aurora \n\n\n\n Show less\n\n\n\n\n\n\n '